## Summary
This notebook reworks code Shanna wrote to process the screening data. It does the following:

1. Load data from DrugBank, ReFRAME, and Broad
2. Creates RDkit molecules from the SMILES, standardizes and sanitizes the molecules by removing salts
3. Calculate Morgan Fingerprints.
4. Save fingerints, molecule names, and source dataset.

In [1]:
import numpy as np
import pandas as pd
import rdkit

from molvs import Standardizer
from rdkit.Chem import PandasTools, SaltRemover, rdMolDescriptors, MolFromSmiles
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')  # suppresses annoying RDKIT errors and warnings

from tqdm import tqdm
tqdm.pandas()

/Users/schu3/.conda/envs/covid/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


### 1. Load Data

In [2]:
drugbank = PandasTools.LoadSDF('../data/screening_data/drugbank.sdf') #auto-sanitize function; don't need to do again
reframe = pd.read_csv('../data/screening_data/reframe.csv', encoding='latin1')
broad = pd.read_csv('../data/screening_data/broad.csv', delimiter="\t")
print('len(drugbank) =', len(drugbank))
print('len(reframe) =', len(reframe))
print('len(broad) =', len(broad))

len(drugbank) = 10746
len(reframe) = 131
len(broad) = 10147


In [3]:
# combine into one dataframe
screening_data = pd.DataFrame(columns=['source', 'name', 'smiles'])
screening_data.source = ['drugbank']*len(drugbank) + ['reframe']*len(reframe) + ['broad']*len(broad)
screening_data.name = pd.concat([drugbank.GENERIC_NAME, reframe.Name, broad.pert_iname], ignore_index=True)
screening_data.smiles = pd.concat([drugbank.SMILES, reframe.SMILES, broad.smiles], ignore_index=True)

print(f"Dropping {screening_data['smiles'].isna().sum()} rows with missing SMILES")
screening_data.dropna(inplace=True)

Dropping 6 rows with missing SMILES


### 2. Create, standardize and sanitize molecules

In [4]:
screening_data['rdkit_mol'] = screening_data['smiles'].progress_apply(MolFromSmiles)
print(f"Dropping {screening_data['rdkit_mol'].isna().sum()} rows which failed molecule creation")
screening_data.dropna(inplace=True)

100%|██████████| 21018/21018 [00:06<00:00, 3346.96it/s]

Dropping 7 rows which failed molecule creation


In [5]:
# standardize molecules
screening_data['rdkit_mol'] = screening_data['rdkit_mol'].progress_apply(Standardizer().standardize)

100%|██████████| 21011/21011 [00:38<00:00, 552.53it/s]


In [6]:
# remove salts
screening_data['rdkit_mol'] = screening_data['rdkit_mol'].progress_apply(SaltRemover.SaltRemover().StripMol)

100%|██████████| 21011/21011 [00:16<00:00, 1288.47it/s]


### 3. Calculate Morgan Fingerprints

In [25]:
bis=[]
def calculate_morgan_fingerprint(mol):
    bi={}
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, useChirality=True, bitInfo=bi)
    bis.append(bi)
    bit_string = fp.ToBitString()
    return np.array([int(char) for char in bit_string], dtype=np.uint8)
screening_data['morgan_fingerprint'] = screening_data['rdkit_mol'].progress_apply(calculate_morgan_fingerprint)

100%|██████████| 21011/21011 [00:13<00:00, 1502.15it/s]


### 4. Save Results

In [28]:
screening_data['bitinfo']=bis

In [30]:
assert not screening_data.isna().values.any()  # confirm clean data
#screening_data.drop(columns=['smiles', 'rdkit_mol']).to_pickle('../processed_data/screening_data_processed.pkl')
screening_data.to_pickle('../processed_data/screening_data_processed.pkl')